<a href="https://colab.research.google.com/github/Kwasjd/Final-Year-Project/blob/master/Image_Classification_TensorFlow_Lite_Model_for_Android_App_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install the following libraries and packages**


In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

!pip install seedir

from google.colab import drive
drive.mount('/content/drive')

# **Import the following Libraries**

In [ ]:
import os

import numpy as np


import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [ ]:
root_path = "/content/"   

In [ ]:
import seedir as sd

image_path = 'drive/MyDrive/food data'
sd.seedir(image_path, style='spaces', indent=4, anystart='- ')
data = DataLoader.from_folder(image_path)

In [ ]:
from pathlib import Path
import imghdr

data_dir = 'drive/MyDrive/food data'
image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

In [ ]:
model = image_classifier.create(data, batch_size = 5)

In [ ]:
loss, accuracy = model.evaluate(data)

In [ ]:
model.export(export_dir='.')

In [ ]:
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

In [ ]:
model = image_classifier.create(train_data, validation_data=validation_data, batch_size=10)

In [ ]:
model.summary()

In [ ]:
loss, accuracy = model.evaluate(test_data)

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

In [ ]:
model.export(export_dir='.')

In [ ]:
model.export(export_dir='.', export_format=ExportFormat.LABEL)

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

In [ ]:
config = QuantizationConfig.for_float16()

In [ ]:
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)

In [ ]:
model = image_classifier.create(train_data, batch_size=10, model_spec=model_spec.get('mobilenet_v2'), validation_data=validation_data)

In [ ]:
loss, accuracy = model.evaluate(test_data)

In [ ]:
inception_v3_spec = image_classifier.ModelSpec(
    uri='https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1')
inception_v3_spec.input_image_shape = [299, 299]

In [ ]:
model = image_classifier.create(train_data, batch_size=8, validation_data=validation_data, epochs=10)

In [ ]:
loss, accuracy = model.evaluate(test_data)